In [82]:
import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lsc
import torch
import h5py

from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
from deeplsd.geometry.viz_2d import plot_images, plot_lines




In [83]:
import torch.onnx 



In [84]:
# Load an image
img = cv2.imread('../assets/images/example.jpg')[:, :, ::-1]
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Model config
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
conf = {
    'detect_lines': True,  # Whether to detect lines or only DF/AF
    'line_detection_params': {
        'merge': False,  # Whether to merge close-by lines
        'filtering': True,  # Whether to filter out lines based on the DF/AF. Use 'strict' to get an even stricter filtering
        'grad_thresh': 3,
        'grad_nfa': True,  # If True, use the image gradient and the NFA score of LSD to further threshold lines. We recommand using it for easy images, but to turn it off for challenging images (e.g. night, foggy, blurry images)
    }
}

# Load the model
ckpt = '../weights/deeplsd_md.tar'
ckpt = torch.load(str(ckpt), map_location='cpu')
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()


H, W = gray_img.shape  # 获取高度和宽度
print(H)
print(W)

480
640


In [85]:
# 假设 net 是你的模型，gray_img 是你的输入图像
dummy_input = torch.randn(1, 1, 480, 640) 
#inputs = {'image': torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.}
inputs= {'image':dummy_input}
#inputs = torch.randn(1,1,224,224)

# 假设 net 是你的模型，gray_img 是你的输入图像
#input_tensor = torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.

# 将输入从字典转换为元组
#inputs = (input_tensor,)

# 将模型导出为 ONNX 格式
torch.onnx.export(net, 
                  inputs, 
                  "DeepLSD.onnx",
                  input_names= ["image"],
                  output_names = ["lines"],
                  verbose = True
                 )

print("Model has been converted to ONNX")


graph():
  %lines : Float(1, 0, strides=[1, 1], requires_grad=0, device=cpu) = onnx::Constant[value=[ CPUFloatType{1,0} ]]()
  return (%lines)

Model has been converted to ONNX


In [5]:
import numpy as np
import torch

# 创建一个 NumPy 数组
np_array = np.array([[1, 2, 3], [4, 5, 6]])

# 将 NumPy 数组转换为 PyTorch 张量
torch_tensor = torch.tensor(np_array, dtype=torch.float)

print("NumPy 数组:")
print(np_array)

print("\nPyTorch 张量:")
print(torch_tensor)


NumPy 数组:
[[1 2 3]
 [4 5 6]]

PyTorch 张量:
tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [78]:
import onnxruntime
from PIL import Image
import torchvision.transforms as transforms

In [67]:
def load_img():
    img = Image.open("../assets/images/example.jpg").convert('L').resize((480, 640))
    img = transforms.ToTensor()(img)
    img = img.unsqueeze(0)
    return img

In [81]:
session = onnxruntime.InferenceSession('DeepLSD.onnx')
# 检查模型的输入信息
input_names = [input.name for input in session.get_inputs()]
print("Model input names:", input_names)

Model input names: []


In [75]:
img = load_img()
session = onnxruntime.InferenceSession('DeepLSD.onnx')
inputs = img.numpy()
#print(inputs)
outs = session.run(None,inputs)
#print(outs)

TypeError: run(): incompatible function arguments. The following argument types are supported:
    1. (self: onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession, arg0: List[str], arg1: Dict[str, object], arg2: onnxruntime.capi.onnxruntime_pybind11_state.RunOptions) -> List[object]

Invoked with: <onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession object at 0x7f78ec48c9f0>, ['145'], array([[[[0.39215687, 0.40392157, 0.40392157, ..., 0.4392157 ,
          0.43529412, 0.44313726],
         [0.39215687, 0.40392157, 0.40392157, ..., 0.4392157 ,
          0.43529412, 0.44313726],
         [0.4       , 0.40392157, 0.40392157, ..., 0.4392157 ,
          0.43529412, 0.44313726],
         ...,
         [0.4       , 0.39607844, 0.3764706 , ..., 0.9411765 ,
          0.95686275, 0.94509804],
         [0.36862746, 0.3647059 , 0.34117648, ..., 0.94509804,
          0.9529412 , 0.9529412 ],
         [0.3019608 , 0.29411766, 0.28627452, ..., 0.9490196 ,
          0.94509804, 0.95294